# Data Quality Assurance - Medical Claims & Membership

This notebook performs comprehensive data quality checks on source tables used in the NRS Beta 2 pipeline:
- **FA_MEDICAL**: Medical claims data
- **FA_MEMBERSHIP**: Member eligibility data (STAGE schema)

## Checks Performed:
1. Column existence validation
2. NULL statistics (count and percentage)
3. Populated statistics (count and percentage)
4. Data type verification
5. Sample value inspection
6. Missing column alerts
7. **Key field population analysis** for medical claims

In [ ]:
# Configuration Parameters
CLIENT_DATA = 'EMBLEM'  # Client identifier

In [ ]:
import logging
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, count, sum as ssum, when, lit
import pandas as pd
from datetime import datetime

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

def get_snowflake_session():
    """Create and return Snowflake session using default connection."""
    return Session.builder.getOrCreate()

session = get_snowflake_session()
logger.info("✓ Snowflake session created successfully")

## Expected Column Definitions

Define the columns required from each source table with their criticality and purpose.

In [ ]:
# FA_MEDICAL expected columns
FA_MEDICAL_COLUMNS = {
    # Core identifiers
    'INDV_ID': {'critical': True, 'description': 'Individual/Member ID', 'key_field': False},
    'CLM_AUD_NBR': {'critical': True, 'description': 'Claim audit number (unique claim ID)', 'key_field': False},
    
    # Service dates - KEY FIELDS
    'SRVC_FROM_DT': {'critical': True, 'description': 'Service from date', 'key_field': True},
    'SRVC_THRU_DT': {'critical': True, 'description': 'Service through date', 'key_field': True},
    'PROCESS_DT': {'critical': True, 'description': 'Claim process date', 'key_field': True},
    'ADMIT_DT': {'critical': False, 'description': 'Hospital admission date', 'key_field': True},
    'DSCHRG_DT': {'critical': False, 'description': 'Hospital discharge date', 'key_field': True},
    
    # Diagnosis codes - KEY FIELDS
    'DIAG_1_CD': {'critical': True, 'description': 'Primary diagnosis code', 'key_field': True},
    'DIAG_2_CD': {'critical': False, 'description': 'Secondary diagnosis code', 'key_field': True},
    'DIAG_3_CD': {'critical': False, 'description': 'Tertiary diagnosis code', 'key_field': True},
    'DIAG_4_CD': {'critical': False, 'description': 'Fourth diagnosis code', 'key_field': True},
    'DIAG_5_CD': {'critical': False, 'description': 'Fifth diagnosis code', 'key_field': True},
    
    # Procedure codes - KEY FIELDS
    'PROC_1_CD': {'critical': False, 'description': 'Primary procedure code', 'key_field': True},
    'PROC_2_CD': {'critical': False, 'description': 'Secondary procedure code', 'key_field': True},
    'PROC_3_CD': {'critical': False, 'description': 'Tertiary procedure code', 'key_field': True},
    'PROC_CD': {'critical': False, 'description': 'CPT/procedure code', 'key_field': False},
    
    # Discharge and DRG - KEY FIELD
    'DSCHRG_STS': {'critical': False, 'description': 'Discharge status', 'key_field': False},
    'DRG': {'critical': False, 'description': 'Diagnosis Related Group code', 'key_field': True},
    'DRG_TYPE': {'critical': False, 'description': 'DRG type (MS-DRG, APR-DRG)', 'key_field': False},
    'DRG_OTLR_FLG': {'critical': False, 'description': 'DRG outlier flag', 'key_field': False},
    'DRG_OTLR_COST': {'critical': False, 'description': 'DRG outlier cost', 'key_field': True},
    
    # Financial - KEY FIELDS
    'SBMT_CHRG_AMT': {'critical': False, 'description': 'Submitted charge amount', 'key_field': True},
    'NET_PD_AMT': {'critical': True, 'description': 'Net paid amount', 'key_field': True},
    
    # Service location and revenue - KEY FIELD
    'PL_OF_SRVC_CD': {'critical': False, 'description': 'Place of service code', 'key_field': False},
    'RVNU_CD': {'critical': False, 'description': 'Revenue code', 'key_field': True},
    
    # Provider information
    'PROV_NPI': {'critical': False, 'description': 'Provider NPI', 'key_field': False},
    'PROV_TIN': {'critical': False, 'description': 'Provider TIN', 'key_field': False},
    'PROV_FULL_NM': {'critical': False, 'description': 'Provider full name', 'key_field': False},
    'PROV_STATE': {'critical': False, 'description': 'Provider state', 'key_field': False},
    'PROV_TYP_CD': {'critical': False, 'description': 'Provider type code', 'key_field': False}
}

# FA_MEMBERSHIP expected columns
FA_MEMBERSHIP_COLUMNS = {
    'INDV_ID': {'critical': True, 'description': 'Individual/Member ID'},
    'GENDER': {'critical': False, 'description': 'Member gender'},
    'BTH_DT': {'critical': True, 'description': 'Member birth date'},
    'MEM_EFF_DT': {'critical': True, 'description': 'Member effective date'},
    'MEM_EXP_DT': {'critical': True, 'description': 'Member expiration date'},
    'PRDCT_CD': {'critical': False, 'description': 'Product code'},
    'STATE': {'critical': False, 'description': 'Member state'},
    'BUS_LINE_CD': {'critical': False, 'description': 'Business line code'}
}

print(f"FA_MEDICAL: {len(FA_MEDICAL_COLUMNS)} expected columns defined")
print(f"FA_MEMBERSHIP: {len(FA_MEMBERSHIP_COLUMNS)} expected columns defined")

## Helper Functions

In [ ]:
def check_column_stats(session, table_name, expected_columns, sample_size=10000):
    """
    Comprehensive column quality check for a Snowflake table.
    
    Args:
        session: Snowpark session
        table_name: Fully qualified table name
        expected_columns: Dict of expected columns with metadata
        sample_size: Number of rows to sample for analysis
        
    Returns:
        dict with results including missing columns, stats, and alerts
    """
    logger.info(f"\n{'='*80}")
    logger.info(f"Analyzing table: {table_name}")
    logger.info(f"{'='*80}")
    
    results = {
        'table_name': table_name,
        'timestamp': datetime.now().isoformat(),
        'missing_columns': [],
        'column_stats': [],
        'alerts': [],
        'summary': {}
    }
    
    try:
        # Load table
        df = session.table(table_name)
        
        # Get total row count
        total_rows = df.count()
        results['summary']['total_rows'] = total_rows
        logger.info(f"Total rows: {total_rows:,}")
        
        if total_rows == 0:
            results['alerts'].append({
                'severity': 'CRITICAL',
                'message': f'Table {table_name} is empty!'
            })
            return results
        
        # Get actual columns from table
        actual_columns = set(df.columns)
        expected_column_names = set(expected_columns.keys())
        
        # Find missing columns
        missing = expected_column_names - actual_columns
        if missing:
            results['missing_columns'] = list(missing)
            for col_name in missing:
                col_info = expected_columns[col_name]
                severity = 'CRITICAL' if col_info['critical'] else 'WARNING'
                results['alerts'].append({
                    'severity': severity,
                    'column': col_name,
                    'message': f"Missing column: {col_name} - {col_info['description']}"
                })
        
        # Analyze each expected column that exists
        for col_name, col_info in expected_columns.items():
            if col_name not in actual_columns:
                continue
                
            logger.info(f"  Analyzing: {col_name}")
            
            # Calculate NULL stats
            null_count = df.filter(col(col_name).is_null()).count()
            null_pct = (null_count / total_rows * 100) if total_rows > 0 else 0
            populated_count = total_rows - null_count
            populated_pct = 100 - null_pct
            
            # Get data type
            col_type = str(df.schema[col_name].datatype)
            
            # Get sample non-null values
            sample_vals = (
                df.filter(col(col_name).is_not_null())
                  .select(col_name)
                  .distinct()
                  .limit(5)
                  .to_pandas()[col_name]
                  .tolist()
            )
            
            # Get distinct count for low-cardinality columns
            distinct_count = None
            if populated_count > 0 and populated_count <= 100000:
                distinct_count = df.select(col_name).distinct().count()
            
            col_stats = {
                'column_name': col_name,
                'description': col_info['description'],
                'critical': col_info['critical'],
                'key_field': col_info.get('key_field', False),
                'data_type': col_type,
                'total_rows': total_rows,
                'null_count': null_count,
                'null_pct': round(null_pct, 2),
                'populated_count': populated_count,
                'populated_pct': round(populated_pct, 2),
                'distinct_count': distinct_count,
                'sample_values': sample_vals
            }
            results['column_stats'].append(col_stats)
            
            # Generate alerts
            if col_info['critical'] and null_pct > 50:
                results['alerts'].append({
                    'severity': 'CRITICAL',
                    'column': col_name,
                    'message': f"Critical column {col_name} has {null_pct:.1f}% NULLs"
                })
            elif col_info['critical'] and null_pct > 10:
                results['alerts'].append({
                    'severity': 'WARNING',
                    'column': col_name,
                    'message': f"Critical column {col_name} has {null_pct:.1f}% NULLs"
                })
            elif null_pct == 100:
                results['alerts'].append({
                    'severity': 'WARNING',
                    'column': col_name,
                    'message': f"Column {col_name} is completely NULL"
                })
        
        # Summary statistics
        results['summary']['expected_columns'] = len(expected_columns)
        results['summary']['found_columns'] = len(expected_column_names & actual_columns)
        results['summary']['missing_columns'] = len(missing)
        results['summary']['critical_alerts'] = len([a for a in results['alerts'] if a['severity'] == 'CRITICAL'])
        results['summary']['warnings'] = len([a for a in results['alerts'] if a['severity'] == 'WARNING'])
        
    except Exception as e:
        results['alerts'].append({
            'severity': 'CRITICAL',
            'message': f"Error analyzing table: {str(e)}"
        })
        logger.error(f"Error: {e}")
    
    return results

In [ ]:
def print_results(results):
    """
    Pretty print the QA results.
    """
    print(f"\n{'='*80}")
    print(f"QA REPORT: {results['table_name']}")
    print(f"Timestamp: {results['timestamp']}")
    print(f"{'='*80}\n")
    
    # Summary
    if 'summary' in results and results['summary']:
        print("SUMMARY:")
        print(f"  Total Rows: {results['summary'].get('total_rows', 'N/A'):,}")
        print(f"  Expected Columns: {results['summary'].get('expected_columns', 0)}")
        print(f"  Found Columns: {results['summary'].get('found_columns', 0)}")
        print(f"  Missing Columns: {results['summary'].get('missing_columns', 0)}")
        print(f"  Critical Alerts: {results['summary'].get('critical_alerts', 0)}")
        print(f"  Warnings: {results['summary'].get('warnings', 0)}")
        print()
    
    # Alerts
    if results['alerts']:
        print(f"\n{'='*80}")
        print("ALERTS:")
        print(f"{'='*80}")
        for alert in sorted(results['alerts'], key=lambda x: (0 if x['severity'] == 'CRITICAL' else 1)):
            severity = alert['severity']
            symbol = '🔴' if severity == 'CRITICAL' else '⚠️'
            print(f"  {symbol} [{severity}] {alert['message']}")
        print()
    else:
        print("✓ No alerts - all checks passed!\n")
    
    # Column Statistics
    if results['column_stats']:
        print(f"\n{'='*80}")
        print("COLUMN STATISTICS:")
        print(f"{'='*80}\n")
        
        # Convert to DataFrame for nice display
        stats_df = pd.DataFrame(results['column_stats'])
        
        # Reorder columns for display
        display_cols = [
            'column_name', 'critical', 'data_type', 
            'populated_count', 'populated_pct',
            'null_count', 'null_pct'
        ]
        
        if 'distinct_count' in stats_df.columns:
            display_cols.append('distinct_count')
        
        display_df = stats_df[display_cols].copy()
        
        # Format for display
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)
        pd.set_option('display.max_colwidth', 30)
        
        print(display_df.to_string(index=False))
        print()
        
        # Print sample values separately for better readability
        print(f"\n{'='*80}")
        print("SAMPLE VALUES:")
        print(f"{'='*80}\n")
        for _, row in stats_df.iterrows():
            samples = row['sample_values'][:3]  # Show first 3 samples
            samples_str = ', '.join([str(s) for s in samples]) if samples else 'No samples'
            print(f"  {row['column_name']:30} {samples_str}")
        print()

In [ ]:
def export_results_to_csv(results, output_file):
    """
    Export QA results to CSV file.
    """
    if results['column_stats']:
        df = pd.DataFrame(results['column_stats'])
        df.to_csv(output_file, index=False)
        logger.info(f"✓ Results exported to {output_file}")
    else:
        logger.warning("No column statistics to export")

## Run QA Checks - FA_MEDICAL Table

In [ ]:
# Check FA_MEDICAL table
fa_medical_table = f"FA_MEDICAL_{CLIENT_DATA}"
fa_medical_results = check_column_stats(
    session, 
    fa_medical_table, 
    FA_MEDICAL_COLUMNS,
    sample_size=10000
)

# Print results
print_results(fa_medical_results)

## Key Field Population Analysis - FA_MEDICAL

Detailed analysis of population rates for key fields used in NRS pipeline.

In [ ]:
# Extract and highlight key field statistics
if fa_medical_results['column_stats']:
    key_fields_df = pd.DataFrame([
        stat for stat in fa_medical_results['column_stats'] 
        if stat.get('key_field', False)
    ])
    
    if not key_fields_df.empty:
        print(f"\n{'='*80}")
        print("KEY FIELD POPULATION RATES - FA_MEDICAL")
        print(f"{'='*80}\n")
        
        # Group by category
        date_fields = ['SRVC_FROM_DT', 'SRVC_THRU_DT', 'PROCESS_DT', 'ADMIT_DT', 'DSCHRG_DT']
        diag_fields = ['DIAG_1_CD', 'DIAG_2_CD', 'DIAG_3_CD', 'DIAG_4_CD', 'DIAG_5_CD']
        proc_fields = ['PROC_1_CD', 'PROC_2_CD', 'PROC_3_CD']
        amount_fields = ['SBMT_CHRG_AMT', 'NET_PD_AMT', 'DRG_OTLR_COST']
        code_fields = ['DRG', 'RVNU_CD']
        
        categories = [
            ('DATE FIELDS', date_fields),
            ('DIAGNOSIS CODES', diag_fields),
            ('PROCEDURE CODES', proc_fields),
            ('AMOUNT FIELDS', amount_fields),
            ('CODE FIELDS', code_fields)
        ]
        
        for cat_name, field_list in categories:
            cat_df = key_fields_df[key_fields_df['column_name'].isin(field_list)]
            if not cat_df.empty:
                print(f"\n{cat_name}:")
                print("-" * 80)
                for _, row in cat_df.iterrows():
                    pop_pct = row['populated_pct']
                    status = '✓' if pop_pct >= 90 else '⚠️' if pop_pct >= 50 else '✗'
                    print(f"  {status} {row['column_name']:20} {pop_pct:6.2f}% populated ({row['populated_count']:,} rows)")
        
        print(f"\n{'='*80}\n")
        
        # Summary statistics
        print("SUMMARY BY CATEGORY:")
        print("-" * 80)
        for cat_name, field_list in categories:
            cat_df = key_fields_df[key_fields_df['column_name'].isin(field_list)]
            if not cat_df.empty:
                avg_pop = cat_df['populated_pct'].mean()
                min_pop = cat_df['populated_pct'].min()
                max_pop = cat_df['populated_pct'].max()
                print(f"  {cat_name:25} Avg: {avg_pop:6.2f}%  Min: {min_pop:6.2f}%  Max: {max_pop:6.2f}%")
        print()

## Run QA Checks - FA_MEMBERSHIP Table

In [ ]:
# Check FA_MEMBERSHIP table
fa_membership_table = f"STAGE.FA_MEMBERSHIP_{CLIENT_DATA}"
fa_membership_results = check_column_stats(
    session, 
    fa_membership_table, 
    FA_MEMBERSHIP_COLUMNS,
    sample_size=10000
)

# Print results
print_results(fa_membership_results)

## Combined Summary Report

In [ ]:
def print_combined_summary(fa_results, fa_membership_results):
    """
    Print a combined summary of both table checks.
    """
    print(f"\n{'='*80}")
    print("COMBINED DATA QA SUMMARY")
    print(f"{'='*80}\n")
    
    tables = [
        ('FA_MEDICAL', fa_results),
        ('PS_MEMBERSHIP', fa_membership_results)
    ]
    
    summary_data = []
    all_critical_alerts = []
    all_warnings = []
    
    for table_name, results in tables:
        if 'summary' in results and results['summary']:
            summary_data.append({
                'Table': table_name,
                'Total Rows': f"{results['summary'].get('total_rows', 0):,}",
                'Expected Cols': results['summary'].get('expected_columns', 0),
                'Found Cols': results['summary'].get('found_columns', 0),
                'Missing Cols': results['summary'].get('missing_columns', 0),
                'Critical Alerts': results['summary'].get('critical_alerts', 0),
                'Warnings': results['summary'].get('warnings', 0)
            })
            
            # Collect critical alerts
            for alert in results['alerts']:
                if alert['severity'] == 'CRITICAL':
                    all_critical_alerts.append(f"{table_name}: {alert['message']}")
                else:
                    all_warnings.append(f"{table_name}: {alert['message']}")
    
    # Display summary table
    summary_df = pd.DataFrame(summary_data)
    print(summary_df.to_string(index=False))
    print()
    
    # Display critical alerts
    if all_critical_alerts:
        print(f"\n🔴 CRITICAL ALERTS ({len(all_critical_alerts)}):")
        for alert in all_critical_alerts:
            print(f"  • {alert}")
    
    # Display warnings
    if all_warnings:
        print(f"\n⚠️  WARNINGS ({len(all_warnings)}):")
        for warning in all_warnings[:10]:  # Show first 10
            print(f"  • {warning}")
        if len(all_warnings) > 10:
            print(f"  ... and {len(all_warnings) - 10} more warnings")
    
    # Overall status
    print(f"\n{'='*80}")
    if not all_critical_alerts:
        print("✓ Overall Status: PASSED (no critical issues)")
    else:
        print(f"✗ Overall Status: FAILED ({len(all_critical_alerts)} critical issues)")
    print(f"{'='*80}\n")

print_combined_summary(fa_medical_results, ps_membership_results)

print_combined_summary(fa_medical_results, fa_membership_results)

In [ ]:
# Export combined results to JSON for programmatic access
import json

combined_results = {
    'timestamp': datetime.now().isoformat(),
    'client': CLIENT_DATA,
    'tables': {
        'FA_MEDICAL': fa_medical_results,
        'FA_MEMBERSHIP': fa_membership_results
    }
}

output_file = f'qa_combined_{CLIENT_DATA}.json'
with open(output_file, 'w') as f:
    json.dump(combined_results, f, indent=2, default=str)

logger.info(f"✓ Combined results exported to {output_file}")

## Recommendations

Based on the QA results above:

### Critical Issues (Must Fix)
- Any missing critical columns must be added to source tables
- Critical columns with >50% NULLs need data quality investigation

### Warnings (Should Review)
- Non-critical columns with high NULL rates may impact specific analyses
- Completely NULL columns should be investigated or removed

### Next Steps
1. Review all critical alerts and coordinate with data engineering team
2. Investigate columns with unexpected NULL rates
3. Validate sample values match expected data formats
4. Re-run QA after any data corrections
5. Schedule regular QA checks (e.g., monthly)

## Baby DRG Frequency Analysis

Identify newborn claims and analyze DRG code utilization patterns. This analysis:
- Identifies babies using revenue codes (REF_NEWBORN_REVCODE) and diagnosis codes (REF_NEWBORN_ICD)
- Calculates DRG frequency by distinct member count
- Shows date ranges for each DRG code

This helps determine which DRG codes are most commonly associated with newborn care.

In [ ]:
# Baby DRG Frequency Analysis
from snowflake.snowpark.functions import count_distinct, min as sfmin, max as sfmax

logger.info("\n" + "="*80)
logger.info("BABY DRG FREQUENCY ANALYSIS")
logger.info("="*80)

# Initialize baby_drg_results as None
baby_drg_results = None

try:
    # Load FA_MEDICAL table
    fa_medical_table = f"FA_MEDICAL_{CLIENT_DATA}"
    df = session.table(fa_medical_table)
    
    logger.info(f"Loading reference tables for newborn identification...")
    
    # Load reference codes for newborn identification
    rev_codes = session.table("SUPP_DATA.REF_NEWBORN_REVCODE").select("CODE").to_pandas()['CODE'].astype(str).tolist()
    icd_codes = session.table("SUPP_DATA.REF_NEWBORN_ICD").select("CODE").to_pandas()['CODE'].astype(str).tolist()
    
    logger.info(f"  - Newborn revenue codes: {len(rev_codes)} codes")
    logger.info(f"  - Newborn ICD codes: {len(icd_codes)} codes")
    
    # Identify baby claims using revenue codes and diagnosis codes
    logger.info(f"\nIdentifying baby claims...")
    
    baby_claims = df.filter(
        # Revenue code match
        col("RVNU_CD").cast("string").isin(rev_codes) |
        # OR any diagnosis code match (DIAG_1_CD through DIAG_5_CD)
        col("DIAG_1_CD").cast("string").isin(icd_codes) |
        col("DIAG_2_CD").cast("string").isin(icd_codes) |
        col("DIAG_3_CD").cast("string").isin(icd_codes) |
        col("DIAG_4_CD").cast("string").isin(icd_codes) |
        col("DIAG_5_CD").cast("string").isin(icd_codes)
    ).select(
        "INDV_ID",
        "SRVC_FROM_DT", 
        "DRG",
        "RVNU_CD",
        "DIAG_1_CD",
        "DIAG_2_CD",
        "DIAG_3_CD",
        "DIAG_4_CD",
        "DIAG_5_CD"
    )
    
    # Count total baby claims
    total_baby_claims = baby_claims.count()
    logger.info(f"  Found {total_baby_claims:,} baby claims")
    
    if total_baby_claims == 0:
        print("\nNo baby claims found. Check if reference tables are populated.")
    else:
        # Analyze DRG frequency for baby claims
        logger.info(f"\nCalculating DRG frequency statistics...")
        
        drg_freq = baby_claims.filter(
            col("DRG").is_not_null()
        ).group_by("DRG").agg(
            count_distinct("INDV_ID").alias("MEMBER_COUNT"),
            sfmin("SRVC_FROM_DT").alias("MIN_DATE"),
            sfmax("SRVC_FROM_DT").alias("MAX_DATE")
        ).order_by(
            col("MEMBER_COUNT").desc()
        )
        
        # Convert to pandas for display and export
        baby_drg_results = drg_freq.to_pandas()
        
        # Display results
        print(f"\n{'='*80}")
        print("DRG CODE FREQUENCY FOR NEWBORN CLAIMS")
        print(f"{'='*80}\n")
        print(f"Total Baby Claims: {total_baby_claims:,}")
        print(f"Unique DRG Codes: {len(baby_drg_results):,}")
        print(f"Unique Members: {baby_claims.select('INDV_ID').distinct().count():,}")
        print()
        
        # Display top DRG codes
        print("TOP DRG CODES BY MEMBER COUNT:")
        print("-" * 80)
        pd.set_option('display.max_rows', 50)
        pd.set_option('display.width', None)
        pd.set_option('display.max_colwidth', None)
        
        # Format the dataframe for display
        display_df = baby_drg_results.copy()
        display_df['MIN_DATE'] = pd.to_datetime(display_df['MIN_DATE']).dt.strftime('%Y-%m-%d')
        display_df['MAX_DATE'] = pd.to_datetime(display_df['MAX_DATE']).dt.strftime('%Y-%m-%d')
        
        print(display_df.to_string(index=False))
        print()
        
        # Summary statistics
        print(f"\n{'='*80}")
        print("SUMMARY STATISTICS:")
        print("-" * 80)
        print(f"  Total DRG codes: {len(baby_drg_results)}")
        print(f"  Average members per DRG: {baby_drg_results['MEMBER_COUNT'].mean():.1f}")
        print(f"  Median members per DRG: {baby_drg_results['MEMBER_COUNT'].median():.1f}")
        print(f"  Most common DRG: {baby_drg_results.iloc[0]['DRG']} ({baby_drg_results.iloc[0]['MEMBER_COUNT']} members)")
        print(f"  Date range: {display_df['MIN_DATE'].min()} to {display_df['MAX_DATE'].max()}")
        print(f"{'='*80}\n")
        
except Exception as e:
    logger.error(f"Error in baby DRG frequency analysis: {e}")
    print(f"\nError: {e}")

## Consolidated Excel Export

Export all QA results to a single Excel file with multiple tabs for easy analysis and sharing.

In [ ]:
# Export all results to a single Excel file with multiple tabs
logger.info("\n" + "="*80)
logger.info("CONSOLIDATED EXCEL EXPORT")
logger.info("="*80)

try:
    output_file = f'data_qa_results_{CLIENT_DATA}.xlsx'
    
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # Tab 1: FA_MEDICAL Column Statistics
        if fa_medical_results and fa_medical_results.get('column_stats'):
            medical_df = pd.DataFrame(fa_medical_results['column_stats'])
            # Drop sample_values column for cleaner export
            if 'sample_values' in medical_df.columns:
                medical_df = medical_df.drop('sample_values', axis=1)
            medical_df.to_excel(writer, sheet_name='FA_MEDICAL_Checks', index=False)
            logger.info(f"  ✓ Exported FA_MEDICAL checks ({len(medical_df)} columns)")
        
        # Tab 2: FA_MEMBERSHIP Column Statistics
        if fa_membership_results and fa_membership_results.get('column_stats'):
            membership_df = pd.DataFrame(fa_membership_results['column_stats'])
            # Drop sample_values column for cleaner export
            if 'sample_values' in membership_df.columns:
                membership_df = membership_df.drop('sample_values', axis=1)
            membership_df.to_excel(writer, sheet_name='FA_MEMBERSHIP_Checks', index=False)
            logger.info(f"  ✓ Exported FA_MEMBERSHIP checks ({len(membership_df)} columns)")
        
        # Tab 3: Baby DRG Frequency
        if baby_drg_results is not None and not baby_drg_results.empty:
            # Format dates for export
            baby_export_df = baby_drg_results.copy()
            baby_export_df['MIN_DATE'] = pd.to_datetime(baby_export_df['MIN_DATE'])
            baby_export_df['MAX_DATE'] = pd.to_datetime(baby_export_df['MAX_DATE'])
            baby_export_df.to_excel(writer, sheet_name='Baby_DRG_Frequency', index=False)
            logger.info(f"  ✓ Exported Baby DRG frequency ({len(baby_export_df)} DRG codes)")
        
        # Tab 4: Summary
        summary_data = []
        
        if fa_medical_results and fa_medical_results.get('summary'):
            summary_data.append({
                'Table': 'FA_MEDICAL',
                'Total_Rows': fa_medical_results['summary'].get('total_rows', 0),
                'Expected_Columns': fa_medical_results['summary'].get('expected_columns', 0),
                'Found_Columns': fa_medical_results['summary'].get('found_columns', 0),
                'Missing_Columns': fa_medical_results['summary'].get('missing_columns', 0),
                'Critical_Alerts': fa_medical_results['summary'].get('critical_alerts', 0),
                'Warnings': fa_medical_results['summary'].get('warnings', 0)
            })
        
        if fa_membership_results and fa_membership_results.get('summary'):
            summary_data.append({
                'Table': 'FA_MEMBERSHIP',
                'Total_Rows': fa_membership_results['summary'].get('total_rows', 0),
                'Expected_Columns': fa_membership_results['summary'].get('expected_columns', 0),
                'Found_Columns': fa_membership_results['summary'].get('found_columns', 0),
                'Missing_Columns': fa_membership_results['summary'].get('missing_columns', 0),
                'Critical_Alerts': fa_membership_results['summary'].get('critical_alerts', 0),
                'Warnings': fa_membership_results['summary'].get('warnings', 0)
            })
        
        if baby_drg_results is not None and not baby_drg_results.empty:
            summary_data.append({
                'Table': 'Baby_DRG_Analysis',
                'Total_Rows': len(baby_drg_results),
                'Expected_Columns': 3,
                'Found_Columns': 3,
                'Missing_Columns': 0,
                'Critical_Alerts': 0,
                'Warnings': 0
            })
        
        if summary_data:
            summary_df = pd.DataFrame(summary_data)
            summary_df.to_excel(writer, sheet_name='Summary', index=False)
            logger.info(f"  ✓ Exported summary tab")
    
    print(f"\n{'='*80}")
    print(f"✓ All QA results exported to: {output_file}")
    print(f"{'='*80}")
    print("\nExcel file contains the following tabs:")
    print("  1. FA_MEDICAL_Checks - Column statistics for medical claims")
    print("  2. FA_MEMBERSHIP_Checks - Column statistics for membership data")
    print("  3. Baby_DRG_Frequency - DRG utilization for newborn claims")
    print("  4. Summary - High-level summary of all checks")
    print(f"{'='*80}\n")
    
except Exception as e:
    logger.error(f"Error creating consolidated Excel file: {e}")
    print(f"\nError: {e}")